In [28]:
# ============================================
# 03_model_training.ipynb  (Google Colab)
# ============================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
import joblib

# ----------------------------------------------------
# 1. Load dataset
# ----------------------------------------------------

df = pd.read_csv("ads_synthetic.csv")
print("Dataset loaded:", df.shape)
df.head()

# ----------------------------------------------------
# 2. Label Encoding
# ----------------------------------------------------

le_gender = LabelEncoder()
le_city = LabelEncoder()
le_adtype = LabelEncoder()
le_target = LabelEncoder()

df["Gender"] = le_gender.fit_transform(df["Gender"])
df["City"] = le_city.fit_transform(df["City"])
df["AdType"] = le_adtype.fit_transform(df["AdType"])
df["Performance"] = le_target.fit_transform(df["Performance"])   # 0,1,2

# ----------------------------------------------------
# 3. Train-Test Split
# ----------------------------------------------------

X = df.drop("Performance", axis=1)
y = df["Performance"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

# ----------------------------------------------------
# 4. Scaling
# ----------------------------------------------------

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ----------------------------------------------------
# 5. Train XGBoost Model  (FIXED)
# ----------------------------------------------------

model = XGBClassifier(
    n_estimators=220,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.7,
    colsample_bytree=0.7,
    eval_metric="mlogloss",
    objective="multi:softprob",   # FIXED
    num_class=3                   # Low/Medium/High
)

print("\nTraining model...")
model.fit(X_train_scaled, y_train)
print("Training complete.")

# ----------------------------------------------------
# 6. Evaluation
# ----------------------------------------------------

y_pred = model.predict(X_test_scaled)

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))

# ----------------------------------------------------
# 7. Save Model Artifacts
# ----------------------------------------------------

joblib.dump(model, "ad_model.pkl")
joblib.dump(scaler, "scaler.pkl")
joblib.dump(le_target, "label_encoder.pkl")

print("\nSaved Files:")
print("✔ ad_model.pkl")
print("✔ scaler.pkl")
print("✔ label_encoder.pkl")


Dataset loaded: (5000, 12)

Training model...
Training complete.

Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1000

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000


Confusion Matrix:

[[1000]]

Saved Files:
✔ ad_model.pkl
✔ scaler.pkl
✔ label_encoder.pkl


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
